In [13]:
!pip install bert
!pip install transformers
!pip install tf-models-official

In [14]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
import random
import pandas as pd
import numpy as np
# from keras import backend as K
from sklearn.metrics import classification_report

from google.colab import drive
drive.mount('/content/drive')

import os

from transformers import BertTokenizer

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Lambda, Dot, Activation, Concatenate, Layer

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
def recall_m(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.squeeze(y_pred, axis=-1), tf.float32)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(tf.squeeze(y_pred, axis=-1), tf.float32)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def dataset_embedding(dataset_path, tokenizer, batch_size=32):
    dataset = pd.read_csv(dataset_path)[["tweet", "sarcastic"]]
    dataset = dataset[dataset['tweet'].notna()]

    tokenized_tweets = [tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweet)) for tweet in dataset['tweet']]

    tweets_with_len = [[tweet, dataset['sarcastic'].iloc[i], len(tweet)] for i, tweet in enumerate(tokenized_tweets)]
    random.Random(42).shuffle(tweets_with_len)

    tweets_with_len.sort(key=lambda x: x[2])
    sorted_tweets_labels = [(tweet_lab[0], tweet_lab[1]) for tweet_lab in tweets_with_len] # remove tweet len
    processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_tweets_labels, output_types=(tf.int32, tf.int32))

    return processed_dataset.padded_batch(batch_size, padded_shapes=((None, ), ()))


def prepare_datasets(train_path, test_path):
    # BertTokenizer = bert.bert_tokenization.FullTokenizer
    bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=True)
    vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

    vocabulary_file_str = vocabulary_file.decode("utf-8")
    do_lower_case_bool = bool(to_lower_case)

    # tokenizer = BertTokenizer(vocabulary_file, to_lower_case)
    tokenizer = BertTokenizer(
        vocab_file=vocabulary_file_str,
        do_lower_case=do_lower_case_bool
    )

    dataset_train = dataset_embedding(train_path, tokenizer)
    dataset_test = dataset_embedding(test_path, tokenizer)

    return dataset_train, dataset_test, tokenizer

In [16]:
debug_flag = int(os.environ.get('KERAS_ATTENTION_DEBUG', 0))

class Attention(object if debug_flag else Layer):
    def __init__(self, units=128, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        input_dim = int(input_shape[-1])
        with K.name_scope(self.name if not debug_flag else 'attention'):
            self.attention_score_vec = Dense(input_dim, use_bias=False, name='attention_score_vec')
            self.h_t = Lambda(lambda x: x[:, -1, :], output_shape=(input_dim,), name='last_hidden_state')
            self.attention_score = Dot(axes=[1, 2], name='attention_score')
            self.attention_weight = Activation('softmax', name='attention_weight')
            self.context_vector = Dot(axes=[1, 1], name='context_vector')
            self.attention_output = Concatenate(name='attention_output')
            self.attention_vector = Dense(self.units, use_bias=False, activation='tanh', name='attention_vector')
        if not debug_flag:
            super(Attention, self).build(input_shape)

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units

    # def __call__(self, inputs, training=None, **kwargs):
    #     if debug_flag:
    #         return self.call(inputs, training, **kwargs)
    #     else:
    #         return super(Attention, self).__call__(inputs, training, **kwargs)

    def __call__(self, inputs, training=None, **kwargs):
        if debug_flag:
            return self.call(inputs, training=training, **kwargs)
        else:
            return super(Attention, self).__call__(inputs, training=training, **kwargs)

    def call(self, inputs, training=None, **kwargs):
        """
        Many-to-one attention mechanism for Keras.
        @param inputs: 3D tensor with shape (batch_size, time_steps, input_dim).
        @param training: not used in this layer.
        @return: 2D tensor with shape (batch_size, units)
        @author: felixhao28, philipperemy.
        """
        if debug_flag:
            self.build(inputs.shape)
        score_first_part = self.attention_score_vec(inputs)
        h_t = self.h_t(inputs)
        score = self.attention_score([h_t, score_first_part])
        attention_weights = self.attention_weight(score)
        context_vector = self.context_vector([inputs, attention_weights])
        pre_activation = self.attention_output([context_vector, h_t])
        attention_vector = self.attention_vector(pre_activation)
        return attention_vector

    def get_config(self):
        """
        Returns the config of a the layer. This is used for saving and loading from a model
        :return: python dictionary with specs to rebuild layer
        """
        config = super(Attention, self).get_config()
        config.update({'units': self.units})
        return config


In [17]:
train_path = '/content/drive/My Drive/Colab Notebooks/6812_gwp/data/Train_Dataset.csv'
test_path = '/content/drive/My Drive/Colab Notebooks/6812_gwp/data/Test_Dataset.csv'

train_data, test_data, tokenizer = prepare_datasets(train_path, test_path)

att_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.vocab), 128),
    tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.3),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(64, activation='relu')),
    tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.3),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(64, activation='relu')),
    tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.3),
    Attention(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# att_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', f1_m])
att_lstm.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy', f1_m]
)
print(att_lstm.summary())

att_lstm.fit(train_data, epochs=10, validation_data = test_data, class_weight={1:4, 0:1})

loss_test, acc_test, f1_test = att_lstm.evaluate(test_data)
print("Loss:", loss_test, "Accuracy:", acc_test, "F1:", f1_test)

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_10 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_5 (TimeDistributed) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_11 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_5 (Attention)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
    217/Unknown 12s 27ms/step - accuracy: 0.4743 - f1_m: 0.2775 - loss: 1.1921

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


217/217 ━━━━━━━━━━━━━━━━━━━━ 13s 33ms/step - accuracy: 0.4736 - f1_m: 0.2778 - loss: 1.1922 - val_accuracy: 0.1429 - val_f1_m: 0.2337 - val_loss: 0.7422
Epoch 2/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.2666 - f1_m: 0.3950 - loss: 1.1663 - val_accuracy: 0.4829 - val_f1_m: 0.2461 - val_loss: 0.7848
Epoch 3/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.5385 - f1_m: 0.4946 - loss: 1.0051 - val_accuracy: 0.5357 - val_f1_m: 0.2492 - val_loss: 0.9936
Epoch 4/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.7510 - f1_m: 0.6100 - loss: 0.8488 - val_accuracy: 0.5129 - val_f1_m: 0.2246 - val_loss: 1.1067
Epoch 5/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.7439 - f1_m: 0.6452 - loss: 0.7287 - val_accuracy: 0.6579 - val_f1_m: 0.2445 - val_loss: 1.0669
Epoch 6/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.8671 - f1_m: 0.7229 - loss: 0.5969 - val_accuracy: 0.5814 - val_f1_m: 0.2117 - val_loss: 1.3790
Epoch 7/10
217/217 ━━━━━━━━━━━━━

In [18]:
all_labels = []
all_preds = []

for x_batch, y_batch in test_data:
    preds = att_lstm.predict(x_batch)
    preds = (preds > 0.5).astype(int).flatten()
    all_preds.extend(preds)
    all_labels.extend(y_batch.numpy())

print(classification_report(all_labels, all_preds))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 471ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 659ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━

In [19]:
att_Blstm = tf.keras.Sequential([
  tf.keras.layers.Embedding(len(tokenizer.vocab), 128),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
  tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(128, activation='relu')),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
  tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(128, activation='relu')),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3)),
  Attention(),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

# att_Blstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', f1_m])
att_Blstm.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy', f1_m]
)
print(att_Blstm.summary())

att_Blstm.fit(train_data, epochs=10, validation_data = test_data, class_weight={1:4, 0:1})

loss_test, acc_test, f1_test = att_Blstm.evaluate(test_data)
print("Loss:", loss_test, "Accuracy:", acc_test, "F1:", f1_test)

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_6 (TimeDistributed) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_5 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_7 (TimeDistributed) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_6 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ attention_6 (Attention)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
    217/Unknown 75s 245ms/step - accuracy: 0.3811 - f1_m: 0.3631 - loss: 1.1897

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


217/217 ━━━━━━━━━━━━━━━━━━━━ 80s 268ms/step - accuracy: 0.3807 - f1_m: 0.3632 - loss: 1.1898 - val_accuracy: 0.1429 - val_f1_m: 0.2337 - val_loss: 0.7371
Epoch 2/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 55s 257ms/step - accuracy: 0.2565 - f1_m: 0.3911 - loss: 1.1679 - val_accuracy: 0.5657 - val_f1_m: 0.1993 - val_loss: 0.7905
Epoch 3/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 56s 258ms/step - accuracy: 0.6654 - f1_m: 0.5566 - loss: 0.9062 - val_accuracy: 0.6086 - val_f1_m: 0.2054 - val_loss: 0.7925
Epoch 4/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 55s 256ms/step - accuracy: 0.8511 - f1_m: 0.6844 - loss: 0.6774 - val_accuracy: 0.5836 - val_f1_m: 0.2174 - val_loss: 1.0865
Epoch 5/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 55s 256ms/step - accuracy: 0.8461 - f1_m: 0.7446 - loss: 0.5128 - val_accuracy: 0.7086 - val_f1_m: 0.1774 - val_loss: 0.9516
Epoch 6/10
217/217 ━━━━━━━━━━━━━━━━━━━━ 55s 256ms/step - accuracy: 0.9168 - f1_m: 0.8058 - loss: 0.3953 - val_accuracy: 0.6286 - val_f1_m: 0.1994 - val_loss: 1.1769
Epoch 7/10
217/217 ━━

In [20]:
all_labels = []
all_preds = []

for x_batch, y_batch in test_data:
    preds = att_Blstm.predict(x_batch)
    preds = (preds > 0.5).astype(int).flatten()
    all_preds.extend(preds)
    all_labels.extend(y_batch.numpy())

print(classification_report(all_labels, all_preds))

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━